In [1]:
import os
from enum import Enum
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from mycolorpy import colorlist as mcp
from z_base import Experiment, Mols, T_ZERO
from scipy.stats import linregress

plt.rc('xtick', labelsize=20)
plt.rc('ytick', labelsize=20)
plt.rc('font', size=20)
plt.rc('axes', titlesize=20)
plt.rc('axes', labelsize=15)
plt.rc('legend', fontsize=15)
plt.rcParams["figure.figsize"] = (10, 7)
plt.rcParams['figure.subplot.left'] = 0.12
plt.rcParams['figure.subplot.right'] = 0.99
plt.rcParams['figure.subplot.top'] = 0.99
plt.rcParams['figure.subplot.bottom'] = 0.1

In [2]:
%matplotlib qt

In [20]:
DF= pd.read_csv(r'Datas\MD_viscosity.csv',index_col=0)
DF.head(3)

,compound,T,Nu,rho,w_mass,w_vol,w,C_but,C_comp,V,ln_Nu,T_1,D,ln_D,D_comp,ln_D_comp,D_but,ln_D_but
0,butanol,286.15,3.72,0.82668,100.0,100.0,100.0,11.153265,0.0,4.499927,1.313724,0.003495,76.922043,4.342792,1.892807e-10,-22.387790,1.892807e-10,-22.387790
1,butanol,286.35,3.70,0.82668,100.0,100.0,100.0,11.153265,0.0,4.475734,1.308333,0.003492,77.391892,4.348882,1.904369e-10,-22.381700,1.904369e-10,-22.381700
2,butanol,286.45,3.70,0.82668,100.0,100.0,100.0,11.153265,0.0,4.475734,1.308333,0.003491,77.418919,4.349231,1.905034e-10,-22.381351,1.905034e-10,-22.381351


In [27]:
class W(Enum):
    w = "w mol  %"
    w_mass = "w mass  %"
    w_vol = "w vol  %"
    T = 'Temperature  C'
    Nu = 'Viscosity  cP'
    ln_Nu = 'ln Viscosity'
    V = 'Kinematic viscosity cCt'
    T_1 = '1000/Temperature  K$^{-1}$'
    #
    D_comp = 'Diffusion   m2/s '
    D_but = 'Diffusion m2/s'
    ln_D_comp = 'log Diffusion'
    ln_D_but = 'log Diffusion '

In [38]:
DF[DF['compound']=='peta']

,compound,T,Nu,rho,w_mass,w_vol,w,C_but,C_comp,V,ln_Nu,T_1,D,ln_D,D_comp,ln_D_comp,D_but,ln_D_but
3406,peta,295.25,1171.428571,1.20,0.0,0.000000,0.000000,0.000000,4.022931,976.190476,7.065978,0.003387,0.252043,-1.378157,1.090071e-12,-27.544778,NaN,NaN
3407,peta,295.65,1132.000000,1.20,0.0,0.000000,0.000000,0.000000,4.022931,943.333333,7.031741,0.003382,0.261175,-1.342565,1.129567e-12,-27.509186,NaN,NaN
3408,peta,295.95,1099.000000,1.20,0.0,0.000000,0.000000,0.000000,4.022931,915.833333,7.002156,0.003379,0.269290,-1.311965,1.164666e-12,-27.478587,NaN,NaN
3409,peta,296.45,1044.000000,1.20,0.0,0.000000,0.000000,0.000000,4.022931,870.000000,6.950814,0.003373,0.283956,-1.258936,1.228094e-12,-27.425558,NaN,NaN
3410,peta,296.55,1031.166667,1.20,0.0,0.000000,0.000000,0.000000,4.022931,859.305556,6.938439,0.003372,0.287587,-1.246230,1.243798e-12,-27.412852,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4514,peta,312.75,85.400000,1.18,5.8,8.278899,19.858185,0.923368,3.726441,72.372881,4.447335,0.003197,3.662178,1.298058,1.583872e-11,-24.868563,9.011458e-12,-25.432524
4515,peta,312.85,85.000000,1.18,5.8,8.278899,19.858185,0.923368,3.726441,72.033898,4.442651,0.003196,3.680588,1.303073,1.591834e-11,-24.863549,9.056760e-12,-25.427510
4516,peta,312.95,82.450000,1.18,5.8,8.278899,19.858185,0.923368,3.726441,69.872881,4.412015,0.003195,3.795634,1.333851,1.641591e-11,-24.832770,9.339850e-12,-25.396731
4517,peta,313.05,83.800000,1.18,5.8,8.278899,19.858185,0.923368,3.726441,71.016949,4.428433,0.003194,3.735680,1.317930,1.615661e-11,-24.848692,9.192323e-12,-25.412652


In [34]:
# Comparation pure
x_i = W.T_1
y_i = W.ln_D_comp
g_i = W.w_mass

compound_filter = (DF["w"] == 100) | (DF["w"] == 0)
df =DF[compound_filter]
data = []

fig, ax = plt.subplots(num="DT")
for (compound, *_), group in df.groupby(by=['compound']):
    y = group[y_i.name]
    T = group[x_i.name]
    ax.scatter(T*1000, y*np.log10(np.e), s=5)

    R = linregress(T, y)
    line_x = np.array([0.00315, 0.0035])
    ax.plot(
        line_x*1000,
        (R.slope * line_x + R.intercept)*np.log10(np.e),
        linewidth=1,
        label=f"{compound:5}",
    )
    print(f'{compound:7}: {np.exp(R.intercept):5.2e} {-R.slope*8.314/1000:5.2f}')
ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

butanol: 7.42e-07 19.72
dmeg   : 4.42e-08 21.09
ocm    : 2.80e-02 58.50
peta   : 2.05e+01 74.99


In [91]:
# Grouped compound Y(x)
compound = Mols.ocm
x_i = W.T_1
y_i = W.ln_D_comp
y2_i = W.ln_D_but

filt = (DF["compound"] == compound.name) & np.isclose(DF['w_mass'],2.39,atol=0.1)
group = DF[filt]

fig, ax = plt.subplots(num="DT")



y = group[y_i.name]
y2=group[y2_i.name]

T = group[x_i.name]

ax.scatter(T*1000, y, s=2,c='green',)

R = linregress(T, y)
line_x = np.array([0.00315, 0.0035])
ax.plot(
    line_x*1000,
    R.slope * line_x + R.intercept,
    linewidth=1,
    c='green',
    label=f"OCM-2",
)

ax.scatter(T*1000, y2, s=2,c='blue',)

R = linregress(T, y2)
line_x = np.array([0.00315, 0.0035])
ax.plot(
    line_x*1000,
    R.slope * line_x + R.intercept,
    linewidth=1,
    c='blue',
    label=f"Butanol-1",
)

ax.set_xlabel(x_i.value)
ax.set_ylabel(y_i.value)

plt.legend()
plt.show()

In [83]:
# Window Y(x)
compound = Mols.ocm
x_i = W.w
y_i = W.ln_D_comp

compound_filter = (DF["compound"] == compound.name)
# | (DF["compound"] == Mols.butanol.name)
grouped = DF[compound_filter]
T=25+T_ZERO



cond = (T - 2.5 <= grouped["T"]) & (grouped["T"] <= T + 2.5)
group2= grouped[cond]
group = group2[[x_i.name, y_i.name]].groupby(by=x_i.name).mean().reset_index()

x = group[x_i.name]
y = group[y_i.name]

fig, ax = plt.subplots(num="DX")
ax.scatter(x, y,  label=f"OCM-2")
line_x=np.array([0,60])
r=linregress(x,y)
ax.plot(line_x,r.slope*line_x+r.intercept,)

# ax.set_title(f"{compound.value.upper()}")
ax.set_xlabel(f"{x_i.value} [%]")
ax.set_ylabel(y_i.value)
plt.legend()
plt.show()

In [6]:
# E
x_i = W.x
y_i = W.ln_D

fig, ax = plt.subplots(num="E")
for (compound, *_), group in DF.groupby(by=["compound"]):
    y = []
    x = []
    for (x_val, *_), sub_group in group.groupby(by=[x_i.name]):
        r = linregress(sub_group[['T_1', y_i.name]])
        y.append(r.slope* 8.314 / 1000)
        x.append(x_val)
    ax.plot([x[0],100], [y[0],-19.73])
    ax.scatter(x, y, label=f"{compound}")

# ax.set_title(f"E {y_i.value}")
ax.set_xlabel(f"{x_i.value}")
ax.set_ylabel("E  kJ/mol")
plt.legend()
plt.show()

AttributeError: x